# Project Euler Problem 145: Reversible Numbers
[Problem description](https://projecteuler.net/problem=145)

Define a function to reverse a number. There are two possible approaches:
- A: Convert the integer to a string, reverse the string, and convert back into an integer
- B: Get each digit of the integer with modulo and dividing

I'm going to write both and see which one runs faster.

In [8]:
def reverseA(num):
    return int(str(num)[::-1])

def reverseB(num):
    reverseNum = 0
    while num > 0:
        reverseNum *= 10
        reverseNum += num%10
        num //= 10 # integer division
    return reverseNum

In [58]:
import timeit
print("ReverseA:", timeit.timeit(stmt=lambda: reverseA(911157498), number=1000000))
print("ReverseB:", timeit.timeit(stmt=lambda: reverseB(911157498), number=1000000))

ReverseA: 0.7485432950197719
ReverseB: 3.3923600430134684


The first method (reversing a string) consistently takes about half the time of the second, so that is the one I'll be using. I have a similar task in checking if every digit of a number is odd, and similarly two different approaches. I'll again code both and perform the same test.

In [59]:
def oddDigitsA(num):
    num = str(num)
    for digit in num:
        if int(digit) % 2 == 0:
            return False
    return True

def oddDigitsB(num):
    while num > 0:
        if num%2==0:
            return False
        num//=10
    return True

import timeit
print("oddDigitsA:", timeit.timeit(stmt=lambda: oddDigitsA(1111111119111574984), number=1000000))
print("oddDigitsB:", timeit.timeit(stmt=lambda: oddDigitsB(1111111119111574984), number=1000000))

oddDigitsA: 4.06040784297511
oddDigitsB: 0.16764603601768613


Interestingly, the second method (integer division) was much faster here, so that is the function I'll be using. 

In [39]:
print("Addition", timeit.timeit("911157498 + 911157498", number = 1000000))

Addition 0.4552810200257227


I've also tested the time addition of two 9 digit numbers takes. To evaluate whether any number is reversible would require
- reversing the number
- adding the number and the reverse of the number
- checking each digit in the sum

It would take 1.3 seconds (0.7 to reverse, 0.45 to add, 0.15 to check odd-ness) to check $10^6$ numbers, thus it would take a thousand times that to check $10^9$ numbers, or a little over 20 minutes. That might look like this:

In [29]:
oddDigits = oddDigitsB
reverse = reverseA
count = 0
for i in range(1, 1000000000):
    if i % 10 != 0: # eliminate leading 0s
        if oddDigits(i + reverse(i)):
            count+=1
print(count)

608720


That produced an answer of $608720$ in 22 minutes and 22 seconds.



However, we can do better.

Every digit of the sum must be odd. This means if the first (left-most) digit of $n$ is odd, then the last digit of reverse($n$) and thus the first digit of $n$ must be even, and vice versa (if the first digit of $n$ is even, the last digit must be odd). This would roughly halve the number of operations.



In [50]:
oddDigits = oddDigitsB
reverse = reverseA
count = 0
for pow10 in range(1, 9): 
    base = 10**pow10
    for leadingDigit in range(1, 10, 2): # odd leading digits
        minNum = base * leadingDigit
        maxNum = minNum+base
        for i in range(minNum + 2, maxNum, 2): # even numbers only
            if i%10!=0:
                if oddDigits(i + reverse(i)):
                    count+=1
    for leadingDigit in range(2, 10, 2): # even leading digits
        minNum = base * leadingDigit
        maxNum = minNum+base
        for i in range(minNum + 1, maxNum, 2): # odd numbers only
            if oddDigits(i + reverse(i)):
                count+=1
print(count)

608720


That produced the same answer in 9 minutes and 11 seconds. Note that this code also skips checking one-digit numbers.

Let's take a closer look at the reversibility of numbers with an odd number of digits. The center digit will always be added to itself, producing an even number. However, you can still have a 3-digit reversible number ($409$ is the example given in the problem) by being careful about the "carry" of an adjacent addition.

A one digit number $n$ can never be reversible, because reverse($n$) = $n$, so $n$ + reverse($n$) = $2n$, so the last digit will always be even.

However, a three digit number $n$ can be reversible if certain conditions are met. I'll label the digits $\underline{n_{-1}}\ \underline{n_{0}}\ \underline{n_{1}}$, where $n_{1}$ is the ones place and $n_{-1}$ is the hundreds. I'll label the sum $n$ + reverse($n$) as $s$ and the digits of the sum $\underline{s_{-2}}\ \underline{s_{-1}}\ \underline{s_{0}}\ \underline{s_{1}}$, where $s_{1}$ is the one's place and $s_{-2}$ is the (potential) thousands place. We can see $s_{1} = n_{1} + n_{-1}$, $s_{0} = n{0} + n{0}$, and $s_{-1} = n_{-1} + n_{1}$, although each digit may need 1 added to it (a carry) if the digit to the right was greater than 10. We can generalize this as 
$s_{i} = \left\{ 
    \begin{array}{lr} 
    n_{i} + n_{-i} & if s_{i + 1} < 10 \\
    n_{i} + n_{-i} + 1 & if s_{i + 1} \geq 10 \\
    \end{array}
\right.$. $n_{0} + n_{0}$ will always be even, so for $s_{0}$ to be odd, a carry must be added to it. Thus, $s_{1}$ must be greater than 10 (i.e. $n_{1} + n_{-1}$ must produce a carry). Because $n_{1} + n_{-1}$ produced a carry, $n_{-1} + n_{1}$ must as well, so $s_{-1} > 10$, so $s_{-2} = 1$. Thus, every reversible 3 digit number must have the first digit and last digit sum to an odd number greater than 10.

However, this doesn't work for five digits. Let there be a five digit reversible number $n$, with digits $\underline{n_{-2}}\ \underline{n_{-1}}\ \underline{n_{0}}\ \underline{n_{1}}\ \underline{n_{2}}$. The same logic as above holds for digits -1 through 1. Because we determined that $s_{-1} > 10$ and there would be a carry on $s_{-2}$, we now know $n_{-2} + n_{2}$ must be even for $s_{-2}$ to be odd. However, this violates one of the first principles we discovered about reversible numbers: that the first and last digits must sum to an odd number (in this case, so that $s_{2}$ can be odd). Thus, there cannot exist a five digit reversible number.

This logic can be continued out to show that there are seven digit reversible numbers, but not nine digit reversible numbers. 

This allows us to make one major optimization. Because there are no nine digit reversible numbers and the problem asks us to find reversible numbers up to 1,000,000,000, we know that we don't need to check numbers 100,000,000-999,999,999 (and 1,000,000,000 could never be reversible anyway), cutting out about 90% of numbers to check (and helpfully, the most computationally expensive numbers to check).

In [53]:
oddDigits = oddDigitsB
reverse = reverseA
count = 0
for i in range(1, 100000000):
    if i % 10 != 0: # eliminate leading 0s
        if oddDigits(i + reverse(i)):
            count+=1
print(count)

608720


The original version took 2 minutes and 37 seconds to run with this change.

In [54]:
oddDigits = oddDigitsB
reverse = reverseA
count = 0
for pow10 in range(1, 8): 
    base = 10**pow10
    for leadingDigit in range(1, 10, 2): # odd leading digits
        minNum = base * leadingDigit
        maxNum = minNum+base
        for i in range(minNum + 2, maxNum, 2): # even numbers only
            if i%10!=0:
                if oddDigits(i + reverse(i)):
                    count+=1
    for leadingDigit in range(2, 10, 2): # even leading digits
        minNum = base * leadingDigit
        maxNum = minNum+base
        for i in range(minNum + 1, maxNum, 2): # odd numbers only
            if oddDigits(i + reverse(i)):
                count+=1
print(count)

608720


The second version took 1 minute and 16 seconds with this change.

I've also modified it to skip checking 5 digit numbers below, but the improvement, if any, is negligible. (Both times I ran it it actually took longer than the code above. If the "if" statement I added to check for that were checked more frequently, it would be possible that the checks involved in skipping 5 digit numbers actually took longer than simply checking each 5 digit number, but the "if" statement is checked 7 times, so I'm chalking this up to computer inconsistencies.)

In [57]:
oddDigits = oddDigitsB
reverse = reverseA
count = 0
for pow10 in range(1, 8): 
    if pow10==4: # skip 5 digit numbers
        continue
    base = 10**pow10
    for leadingDigit in range(1, 10, 2): # odd leading digits
        minNum = base * leadingDigit
        maxNum = minNum+base
        for i in range(minNum + 2, maxNum, 2): # even numbers only
            if i%10!=0:
                if oddDigits(i + reverse(i)):
                    count+=1
    for leadingDigit in range(2, 10, 2): # even leading digits
        minNum = base * leadingDigit
        maxNum = minNum+base
        for i in range(minNum + 1, maxNum, 2): # odd numbers only
            if oddDigits(i + reverse(i)):
                count+=1
print(count)

608720
